In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim
import os


Set up

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10
BGL = 1

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

SHOW_GW=True
ALL_TARGETS = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

In [ ]:
if BUILD_ARRAYS:
    for target in ALL_TARGETS:
        # if target=='hard_time' or target=='hard_gamma_inner':
        #     continue
        detstats.build_ratio_arrays(target, bgl=1, npsrs=NPSRS) #, gw_only=True)

# 1Col White Noise + GW Only

In [ ]:
INCLUDE_GW = True

In [ ]:
ylabel = plot.LABEL_EVSS
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col0p0 = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]

col_gw = [
    '#595959', '#595959',
    '#595959', '#595959',
    '#595959', '#595959', 
]

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    xx, yy0p0, = detstats.get_ratio_arrays(targets[ii], bgl=BGL)
    yy0p0 *= 0.5

    col=col0p0[ii]
    for pp in [68, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
        ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

    if INCLUDE_GW:
        if ii<4:
            xx_gw, yy_gw = detstats.get_ratio_arrays(targets[ii],  gw_only=True, bgl=BGL)
            yy_gw *= 0.5
            print(f"{ii=}, {yy_gw.shape}")
        else:
            xx_gw = xx
            yy_gw = yy_gw[int(NVARS/2)]
            yy_gw = np.repeat(yy_gw, 21).reshape(NREALS*NSKIES, NVARS)
            yy_gw = np.swapaxes(yy_gw, 0, 1).reshape(NVARS, NREALS, NSKIES,)
            print(f"{ii=}, {yy_gw.shape}")

        for pp in [68]:
            med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.95, color=col_gw[ii], linestyle='-.')
            ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.3,)
        print(f"{ii=}, yy_gw.shape")

    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')       
plt.subplots_adjust(wspace=0.05, hspace=0.35)


if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/evss'
    savename = savename+f'/evss_vs_var{NVARS}_wn_bgl{BGL}_p{NPSRS}_68-95.png'
    fig.savefig(savename, dpi=100, bbox_inches='tight')

# Red Noise

In [ ]:
cmap_Oranges = plot.truncate_colormap('Oranges', 0.4, 1)
cmap_Reds = plot.truncate_colormap('Reds', 0.4, 1)
cmap_Purples = plot.truncate_colormap('Purples', 0.4, 1)
cmap_Greys = plot.truncate_colormap('Greys', 0.4, 0.9)

In [ ]:
ylabel = plot.LABEL_EVSS
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']

color_wn = 'black' 
lines_wn = ':'
cmaps_rn = [cmap_Purples, cmap_Reds, ] #, cmap_Purples]
lines_rn = ['-.', '-', '--']

gammas = [ -1.5,-3.0]
red2whites = [0.01, 1.0, 100.0]

handles_gamma = []
labels_gamma = []

handles_red2white = []
labels_red2white = []

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    # white noise
    xx, yy, = detstats.get_ratio_arrays(targets[ii], bgl=BGL)
    yy *= 0.5
    col=color_wn
    for pp in [68,]:
        med, *conf = np.percentile(yy, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=color_wn, linestyle=lines_wn,
                lw=2, )
        ax.fill_between(xx, *conf, color=color_wn, alpha=0.15)

    # red noise
    for gg, gamma in enumerate(gammas):
        colors = cmaps_rn[gg](np.linspace(0,1,len(red2whites)))
        for rr, red2white in enumerate(red2whites):
            xx, yy = detstats.get_ratio_arrays(targets[ii], bgl=BGL,
                                               red2white=red2white, red_gamma=gamma)
            hh, = ax.plot(xx, np.median(yy, axis=(-2,-1)), alpha=0.9, 
                         color=colors[rr], linestyle=lines_rn[rr])
            # if rr==1:
            #     for pp in [68,]:
            #         lo, hi = np.percentile(yy, [50-pp/2, 50+pp/2], axis=(-2,-1))
            #         ax.fill_between(xx, lo, hi, color=colors[rr], alpha=0.15)

            if ii==0 and lines_rn[rr] == '-':
                handles_gamma.append(hh)
                labels_gamma.append(f"{gamma:.1f}")
                print(gg,rr, colors[rr])
            if ii==0 and gg==1:
                handles_red2white.append(hh)
                labels_red2white.append(f"{red2white:}")

    ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
    ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=np.arange(1.0, 10.0) * 0.1))
      

    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')       

        # med, *conf = np.percentile(yy1p5, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        # ax.plot(xx, med, alpha=0.9, color=col1p5[ii])
        # ax.fill_between(xx, *conf, color=col1p5[ii], alpha=0.25)

        # med, *conf = np.percentile(yy3p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        # ax.plot(xx, med, alpha=0.9, color=col3p0[ii])
        # ax.fill_between(xx, *conf, color=col3p0[ii], alpha=0.25)

# handles = [mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=0.0$', color = col0p0[0]),
#            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-1.5$', color = col1p5[0]),
#            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-3.0$', color = col3p0[0])]
# axs[0,0].legend(handles=handles, loc='upper left', ncol=2)
plt.subplots_adjust(wspace=0.05, hspace=0.35)
leg1 = fig.legend(handles=handles_gamma, labels=labels_gamma, title='$\gamma_\mathrm{RN}$',
                  bbox_to_anchor=(0.18,-0.02), loc='lower left', ncols=len(handles_gamma))
leg2 = fig.legend(handles=handles_red2white, labels=labels_red2white, title='$Q$', #'$S_\mathrm{RN}(f_\mathrm{ref})/S_\mathrm{WN}(f_\mathrm{ref})$',
                  bbox_to_anchor=(0.5, -0.02), loc='lower left', ncols=len(handles_red2white))

# fig.tight_layout()

if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/evss'
    savename = savename+f'/evss_rn_bgl{BGL}.png'
    fig.savefig(savename, dpi=100, bbox_inches='tight')

In [ ]:
xx, yy_100, = detstats.get_ratio_arrays('gsmf_phi0', bgl=BGL,
                                       red2white=100, red_gamma=-3)
xx, yy_1, = detstats.get_ratio_arrays('gsmf_phi0', bgl=BGL,
                                       red2white=1, red_gamma=-3)
xx, yy_p01, = detstats.get_ratio_arrays('gsmf_phi0', bgl=BGL,
                                       red2white=0.01, red_gamma=-3)
xx, yy_wt, = detstats.get_ratio_arrays('gsmf_phi0', bgl=BGL,)

In [ ]:
print(yy_wt.shape
      )

In [ ]:
fig, ax = plot.figax(xlabel='gsmf_phi0', ylabel='evss', xscale='linear')

labels = ['R2W=100', 'R2W=1', 'R2W=0.01', 'WN']
handles = []
for yy in [yy_100, yy_1, yy_p01, yy_wt]:
    yy *= 0.5
    hh = plot.draw_med_conf(ax,xx,yy.reshape(NVARS, NREALS*NSKIES), fracs=[.68,] )
    handles.append(hh)

ax.legend(handles=handles, labels=labels)

In [ ]:
fig, ax = plot.figax(xlabel='gsmf_phi0', ylabel='evss', xscale='linear')

xx, yy, = detstats.get_ratio_arrays('gsmf_phi0', bgl=BGL, nexcl=3)
yy  *= 0.5
h1 = plot.draw_med_conf_color(ax,xx,yy.reshape(NVARS, NREALS*NSKIES), fracs=[0.5,0.95], color='tab:blue' )
h2 = plot.draw_med_conf_color(ax,xx,yy.reshape(NVARS, NREALS*NSKIES), fracs=[0.5,0.99], color='tab:green')

print(yy.shape)
_, y2 = detstats.get_ratio_arrays('gsmf_phi0', bgl=BGL)
y2*=0.5
for pp in [95, 99]:
    med, lo, hi = np.percentile(y2, (50, 50-pp/2, 50+pp/2), axis=(1,2))
    h3, = ax.plot(xx, med, linestyle='-', color='grey')

    ls = '--' if pp==95 else ':'
    ax.plot(xx, hi, linestyle=ls, color='grey', alpha=0.9)
    ax.plot(xx, lo, linestyle=ls, color='grey', alpha=0.9)
plot.draw_med_conf_color(ax,xx,y2.reshape(NVARS, NREALS*NSKIES), fracs=[0.5,], color='grey')

labels = ['$N_\mathrm{excl}=3$, 95% CI', '$N_\mathrm{excl}=3$, 99% CI', '$N_\mathrm{excl}=0$']
ax.legend(handles=[h1,h2,h3], labels=labels, loc='lower left')

In [ ]:
target = 'hard_time'

fig, ax = plot.figax(xlabel=target, ylabel='evss', xscale='linear')

xx, yy, = detstats.get_ratio_arrays(target, bgl=BGL, nexcl=3)
yy  *= 0.5
h1 = plot.draw_med_conf_color(ax,xx,yy.reshape(NVARS, NREALS*NSKIES), fracs=[0.5,0.95], color='tab:blue' )
h2 = plot.draw_med_conf_color(ax,xx,yy.reshape(NVARS, NREALS*NSKIES), fracs=[0.5,0.99], color='tab:green')

print(yy.shape)
_, y2 = detstats.get_ratio_arrays(target, bgl=BGL)
y2*=0.5
for pp in [95, 99]:
    med, lo, hi = np.percentile(y2, (50, 50-pp/2, 50+pp/2), axis=(1,2))
    h3, = ax.plot(xx, med, linestyle='-', color='grey')

    ls = '--' if pp==95 else ':'
    ax.plot(xx, hi, linestyle=ls, color='grey', alpha=0.9)
    ax.plot(xx, lo, linestyle=ls, color='grey', alpha=0.9)
plot.draw_med_conf_color(ax,xx,y2.reshape(NVARS, NREALS*NSKIES), fracs=[0.5,], color='grey')

labels = ['$N_\mathrm{excl}=3$, 95% CI', '$N_\mathrm{excl}=3$, 99% CI', '$N_\mathrm{excl}=0$']
ax.legend(handles=[h1,h2,h3], labels=labels, loc='lower left')